# MPPT Controller for Photovoltaic Systems

## Introduction

This notebook demonstrates Maximum Power Point Tracking (MPPT) algorithms for solar photovoltaic (PV) systems.

### What is MPPT?

MPPT is a technique used to extract maximum power from solar panels under varying environmental conditions (temperature, irradiance). The maximum power point (MPP) changes with:
- Solar irradiance levels
- Temperature
- Partial shading

### MPPT Algorithms Implemented:

1. **Perturb and Observe (P&O)**: Simple and widely used algorithm that perturbs the operating voltage and observes the power change
2. **Incremental Conductance (InCond)**: More precise algorithm that uses the relationship dI/dV = -I/V at MPP
3. **Constant Voltage (CV)**: Maintains voltage at a fixed percentage of open-circuit voltage
4. **Fuzzy Logic**: Advanced algorithm using fuzzy logic control

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from mppt_controller import create_mppt_controller, PerturbAndObserve, IncrementalConductance
from solar_panel import SolarPanel, SolarArray, create_standard_panel
from mppt_simulation import MPPTSimulator, BoostConverter

# Configure matplotlib for better plots
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## Part 1: Solar Panel Characteristics

First, let's examine the I-V and P-V characteristics of a solar panel under different conditions.

In [ ]:
# Create a solar panel model
panel = create_standard_panel('generic')

print("Solar Panel Specifications at STC (25°C, 1000 W/m²):")
print(f"  Open Circuit Voltage (Voc): {panel.V_oc:.2f} V")
print(f"  Short Circuit Current (Isc): {panel.I_sc:.2f} A")
print(f"  Max Power Voltage (Vmp): {panel.V_mp:.2f} V")
print(f"  Max Power Current (Imp): {panel.I_mp:.2f} A")
print(f"  Max Power (Pmax): {panel.P_max:.2f} W")
print(f"  Number of cells in series: {panel.N_s}")

In [ ]:
# Plot I-V and P-V curves under different irradiance levels
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

irradiance_levels = [1000, 800, 600, 400, 200]
temperature = 25.0

for G in irradiance_levels:
    # Get I-V curve
    V, I = panel.get_iv_curve(temperature, G, num_points=100)
    ax1.plot(V, I, linewidth=2, label=f'{G} W/m²')
    
    # Get P-V curve
    V, P = panel.get_pv_curve(temperature, G, num_points=100)
    ax2.plot(V, P, linewidth=2, label=f'{G} W/m²')
    
    # Mark MPP
    v_mpp, i_mpp, p_mpp = panel.find_mpp(temperature, G)
    ax1.plot(v_mpp, i_mpp, 'ko', markersize=6)
    ax2.plot(v_mpp, p_mpp, 'ko', markersize=6)

ax1.set_xlabel('Voltage (V)', fontsize=12)
ax1.set_ylabel('Current (A)', fontsize=12)
ax1.set_title('I-V Characteristics (T = 25°C)', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Voltage (V)', fontsize=12)
ax2.set_ylabel('Power (W)', fontsize=12)
ax2.set_title('P-V Characteristics (T = 25°C)', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nThe black dots indicate the Maximum Power Points (MPP) for each irradiance level.")

In [ ]:
# Plot I-V and P-V curves under different temperature levels
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

temperatures = [0, 25, 50, 75]
irradiance = 1000.0

for T in temperatures:
    # Get I-V curve
    V, I = panel.get_iv_curve(T, irradiance, num_points=100)
    ax1.plot(V, I, linewidth=2, label=f'{T}°C')
    
    # Get P-V curve
    V, P = panel.get_pv_curve(T, irradiance, num_points=100)
    ax2.plot(V, P, linewidth=2, label=f'{T}°C')
    
    # Mark MPP
    v_mpp, i_mpp, p_mpp = panel.find_mpp(T, irradiance)
    ax1.plot(v_mpp, i_mpp, 'ko', markersize=6)
    ax2.plot(v_mpp, p_mpp, 'ko', markersize=6)

ax1.set_xlabel('Voltage (V)', fontsize=12)
ax1.set_ylabel('Current (A)', fontsize=12)
ax1.set_title('I-V Characteristics (G = 1000 W/m²)', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Voltage (V)', fontsize=12)
ax2.set_ylabel('Power (W)', fontsize=12)
ax2.set_title('P-V Characteristics (G = 1000 W/m²)', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nNote: Higher temperature reduces the output voltage and power.")

## Part 2: MPPT Algorithm - Perturb and Observe (P&O)

The P&O algorithm works by:
1. Measuring the current voltage and power
2. Perturbing (changing) the voltage slightly
3. Observing if power increased or decreased
4. Adjusting the perturbation direction accordingly

In [ ]:
# Simulate P&O algorithm
print("Simulating Perturb and Observe (P&O) Algorithm...\n")

sim_po = MPPTSimulator(panel, mppt_algorithm='P&O')
sim_po.run_simulation(duration=5.0, dt=0.01, temperature=25.0, irradiance=1000.0)

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Voltage
axes[0, 0].plot(sim_po.time, sim_po.voltage_history, 'b-', linewidth=2)
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Voltage (V)')
axes[0, 0].set_title('PV Operating Voltage')
axes[0, 0].grid(True, alpha=0.3)

# Current
axes[0, 1].plot(sim_po.time, sim_po.current_history, 'r-', linewidth=2)
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Current (A)')
axes[0, 1].set_title('PV Current')
axes[0, 1].grid(True, alpha=0.3)

# Power
axes[1, 0].plot(sim_po.time, sim_po.power_history, 'g-', linewidth=2)
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Power (W)')
axes[1, 0].set_title('PV Power Output')
axes[1, 0].grid(True, alpha=0.3)

# Tracking Efficiency
axes[1, 1].plot(sim_po.time, sim_po.efficiency_history, 'c-', linewidth=2)
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Efficiency (%)')
axes[1, 1].set_title('MPPT Tracking Efficiency')
axes[1, 1].set_ylim([90, 101])
axes[1, 1].grid(True, alpha=0.3)

fig.suptitle('P&O Algorithm Performance', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Average Power: {np.mean(sim_po.power_history):.2f} W")
print(f"Average Tracking Efficiency: {np.mean(sim_po.efficiency_history):.2f}%")

## Part 3: MPPT Algorithm - Incremental Conductance

The Incremental Conductance algorithm uses the derivative of power with respect to voltage:

$$\frac{dP}{dV} = \frac{d(VI)}{dV} = I + V\frac{dI}{dV}$$

At the MPP: $\frac{dP}{dV} = 0$, therefore: $\frac{dI}{dV} = -\frac{I}{V}$

In [ ]:
# Simulate Incremental Conductance algorithm
print("Simulating Incremental Conductance Algorithm...\n")

sim_incond = MPPTSimulator(panel, mppt_algorithm='InCond')
sim_incond.run_simulation(duration=5.0, dt=0.01, temperature=25.0, irradiance=1000.0)

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Voltage
axes[0, 0].plot(sim_incond.time, sim_incond.voltage_history, 'b-', linewidth=2)
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Voltage (V)')
axes[0, 0].set_title('PV Operating Voltage')
axes[0, 0].grid(True, alpha=0.3)

# Current
axes[0, 1].plot(sim_incond.time, sim_incond.current_history, 'r-', linewidth=2)
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Current (A)')
axes[0, 1].set_title('PV Current')
axes[0, 1].grid(True, alpha=0.3)

# Power
axes[1, 0].plot(sim_incond.time, sim_incond.power_history, 'g-', linewidth=2)
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Power (W)')
axes[1, 0].set_title('PV Power Output')
axes[1, 0].grid(True, alpha=0.3)

# Tracking Efficiency
axes[1, 1].plot(sim_incond.time, sim_incond.efficiency_history, 'c-', linewidth=2)
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Efficiency (%)')
axes[1, 1].set_title('MPPT Tracking Efficiency')
axes[1, 1].set_ylim([90, 101])
axes[1, 1].grid(True, alpha=0.3)

fig.suptitle('Incremental Conductance Algorithm Performance', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Average Power: {np.mean(sim_incond.power_history):.2f} W")
print(f"Average Tracking Efficiency: {np.mean(sim_incond.efficiency_history):.2f}%")

## Part 4: Algorithm Comparison

Let's compare the performance of P&O and Incremental Conductance algorithms under varying irradiance conditions.

In [ ]:
# Simulate both algorithms with varying irradiance
print("Comparing MPPT Algorithms under Varying Conditions...\n")

algorithms = ['P&O', 'InCond']
colors = ['b', 'r']
results = {}

for algo in algorithms:
    sim = MPPTSimulator(panel, mppt_algorithm=algo)
    sim.run_simulation(duration=10.0, dt=0.01, variable_conditions=True)
    results[algo] = sim

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Power comparison
for i, algo in enumerate(algorithms):
    axes[0, 0].plot(results[algo].time, results[algo].power_history, 
                   color=colors[i], linewidth=2, label=algo)
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Power (W)')
axes[0, 0].set_title('Power Output Comparison')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Efficiency comparison
for i, algo in enumerate(algorithms):
    axes[0, 1].plot(results[algo].time, results[algo].efficiency_history,
                   color=colors[i], linewidth=2, label=algo)
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Efficiency (%)')
axes[0, 1].set_title('Tracking Efficiency Comparison')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Voltage comparison
for i, algo in enumerate(algorithms):
    axes[1, 0].plot(results[algo].time, results[algo].voltage_history,
                   color=colors[i], linewidth=2, label=algo)
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Voltage (V)')
axes[1, 0].set_title('Operating Voltage Comparison')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Duty cycle comparison
for i, algo in enumerate(algorithms):
    axes[1, 1].plot(results[algo].time, results[algo].duty_cycle_history,
                   color=colors[i], linewidth=2, label=algo)
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Duty Cycle')
axes[1, 1].set_title('Control Signal Comparison')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

fig.suptitle('MPPT Algorithm Comparison (Variable Irradiance)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Print statistics
print("\nPerformance Statistics:")
print("=" * 50)
for algo in algorithms:
    print(f"\n{algo} Algorithm:")
    print(f"  Average Power: {np.mean(results[algo].power_history):.2f} W")
    print(f"  Average Efficiency: {np.mean(results[algo].efficiency_history):.2f}%")
    print(f"  Std Dev Efficiency: {np.std(results[algo].efficiency_history):.2f}%")

## Part 5: Solar Array Configuration

Let's explore how multiple panels can be configured in series and parallel to create a solar array.

In [ ]:
# Create solar arrays with different configurations
configs = [
    (1, 1, 'Single Panel'),
    (2, 1, '2 Series x 1 Parallel'),
    (1, 2, '1 Series x 2 Parallel'),
    (4, 3, '4 Series x 3 Parallel')
]

print("Solar Array Configurations:")
print("=" * 70)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (N_s, N_p, label) in enumerate(configs):
    array = SolarArray(panel, N_series=N_s, N_parallel=N_p)
    
    print(f"\n{label}:")
    print(f"  Voc: {array.V_oc:.2f} V")
    print(f"  Isc: {array.I_sc:.2f} A")
    print(f"  Pmax: {array.P_max:.2f} W")
    
    # Plot P-V curve
    V, P = array.get_pv_curve(25.0, 1000.0, num_points=100)
    axes[idx].plot(V, P, linewidth=2)
    
    # Mark MPP
    v_mpp, i_mpp, p_mpp = array.find_mpp(25.0, 1000.0)
    axes[idx].plot(v_mpp, p_mpp, 'ro', markersize=10, label=f'MPP: {p_mpp:.0f} W')
    
    axes[idx].set_xlabel('Voltage (V)')
    axes[idx].set_ylabel('Power (W)')
    axes[idx].set_title(label)
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Conclusion

This notebook demonstrated:

1. **PV Panel Characteristics**: How voltage, current, and power vary with irradiance and temperature
2. **MPPT Algorithms**: Implementation and performance of P&O and Incremental Conductance algorithms
3. **Algorithm Comparison**: Performance comparison under varying environmental conditions
4. **Solar Arrays**: Series and parallel configurations of solar panels

### Key Takeaways:

- MPPT is essential for extracting maximum power from solar panels
- Different algorithms have trade-offs between complexity, speed, and accuracy
- Environmental conditions significantly affect PV performance
- Proper array configuration is important for system design